# Python - C++ integration with GitHub Actions and Docker

This notebook addresses how a <code>C++</code> class could be embedded on a <code>Python</code> script. 

Resulting code is executed on a CI/CD pipeline which makes use of  <code>GitHub Actions</code> and <code>Docker</code> hub. 

The test case shown here is a simple C++ class with a member function template which prints a given array of type T and size narray. The specific instantiation of the class is required so that the compiler can generate the appropriate Python interface. 

Python C++ interface is implemented through swig, a software tool that connects programs written in C and C++ with a variety of high-level programming languages.

swig includes library modules for manipulating pointers and arrays (an array name is a constant pointer to the first element of the array).

In the case discussed here, two libraries have been used to generate wrappers around C/C++ arrays, carrays.i and numpy.i. 

The first one is a basic interface file which defines typemaps that assist in wrapping ordinary pointers as arrays. On the other hand, the numpy.i library provides support for converting an C or C++ array to and from a NumPy array. Interfacing is applied via %array_class and %apply macros. 

%array_class(ArrayType,ArrayName) wraps a pointer of type ArrayType inside a class-based interface ArrayName. ArrayType is restricted to a simple type name like int or float. 

%apply directive implements the typemaps defined by numpy.i.  One of these typemaps is the signature (double* IN_ARRAY1, int DIM1). It suggest that the double* argument IN_ARRAY1 is an one-dimensional input array, and where the int argument DIM1 represents the size of that dimension. 



[1]  C arrays and pointers, SWIG-4.0, http://www.swig.org/doc.html 

[2] numpy.i: a SWIG Interface File for NumPy, https://numpy.org/doc/stable/reference/swig.interface-file.html 


### Useful information 

In [2]:
import os 
print("PWD:",os.getcwd())

import datetime
print("TODAY:", datetime.datetime.now())

PWD: /Users/poderozita/Downloads/home/jovyan/work
TODAY: 2021-11-19 18:49:59.161271


In [2]:
%%bash 
cmake --version 

cmake version 3.21.3

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [3]:
%%bash 
g++ --version

g++ (GCC) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
%%bash 
swig -version


SWIG Version 4.0.2

Compiled with /home/conda/feedstock_root/build_artifacts/swig_1614618251926/_build_env/bin/x86_64-conda-linux-gnu-c++ [x86_64-conda-linux-gnu]

Configured options: +pcre

Please see http://www.swig.org for reporting bugs and further information


In [5]:
%%bash 
python --version

Python 3.9.7


In [6]:
import numpy 
numpy.get_include()

'/opt/conda/lib/python3.9/site-packages/numpy/core/include'

### Test case 

In [7]:
%%writefile simpleExample.hpp 

void say_hello();

class SimpleExample 
{
  public:
    SimpleExample(){};
   ~SimpleExample(){};
        
    void ShowArray(double* array, int narray); 
    void ShowArray(int* array, int narray); 
    
  private:
    template<class T>
    void ShowArray(T* array, int narray);     
};

Writing simpleExample.hpp


In [8]:
%%writefile simpleExample.cpp
#include <iostream>
#include "simpleExample.hpp"

void say_hello()
{
    std::cout<<"Hello from swig-example:" << std::endl;
}

void 
SimpleExample::ShowArray(double* array, int narray)
{
    this->ShowArray<double>(array, narray);     
}


void 
SimpleExample::ShowArray(int* array, int narray)
{
    this->ShowArray<int>(array, narray); 
}


template<class T>
void 
SimpleExample::ShowArray(T* array, int narray)
{
    std::cout<<" [ "; 
    for(int i=0; i<narray; i++)
    {
        std::cout<< array[i] <<" "; 
    }
    std::cout<<"] "<< std::endl; 
}

Writing simpleExample.cpp


In [9]:
%%writefile test.py
import numpy as np
import swig_example

swig_example.say_hello()
E = swig_example.SimpleExample()

narray = 4
array1 = np.arange(narray) + 1
E.ShowArray(array1)

array2 = swig_example.iArray(narray)
for i in range(narray): array2[i] = i+1 
E.ShowArray(array2,narray)

Writing test.py


### Configuring *swig* 

In [10]:
%%writefile swigConfigurationFile.i  
## 
%module swig_example

%{
    #define SWIG_FILE_WITH_INIT
    #include "simpleExample.hpp"
%}

## ShowArray(int* array, int narray)
%include "carrays.i"
%array_class(int, iArray)

## 
%include "numpy.i"
%init 
%{
    import_array();
%}

## ShowArray(double* array, int narray)
%apply (double* IN_ARRAY1, int DIM1) {(double* array, int narray)}

%include "simpleExample.hpp"

Writing swigConfigurationFile.i


### If *numpy.i* is missing, activate this cell 

In [11]:
import re
import requests
import numpy

np_version = re.compile(r'(?P<MAJOR>[0-9]+)\.'
                        '(?P<MINOR>[0-9]+)') \
                        .search(numpy.__version__)
np_version_string = np_version.group()
np_version_info = {key: int(value)
                   for key, value in np_version.groupdict().items()}

np_file_name = 'numpy.i'
np_file_url = 'https://raw.githubusercontent.com/numpy/numpy/maintenance/' + \
              np_version_string + '.x/tools/swig/' + np_file_name
if(np_version_info['MAJOR'] == 1 and np_version_info['MINOR'] < 9):
    np_file_url = np_file_url.replace('tools', 'doc')

chunk_size = 8196
with open(np_file_name, 'wb') as file:
    for chunk in requests.get(np_file_url,
                              stream=True).iter_content(chunk_size):
        file.write(chunk)

### Configuring *CMake* 

In [12]:
%%writefile CMakeListsFromPynb.txt 
## 
cmake_minimum_required(VERSION 3.19.3)
project(text CXX)
set(CMAKE_CXX_STANDARD 11)
set(CMAKE_CXX_STANDARD_REQUERIRED ON)

## 
include_directories(./)
add_library(LibraryName SHARED simpleExample.cpp)

#add_executable(tobetested y.cxx)
#target_link_libraries(tobetested fileName)

#include_directories(/Users/poderozita/anaconda3/envs/jmake2021/lib/python3.7/site-packages/numpy/core/include)

## Wrapper.Python
find_package(Python3 COMPONENTS Interpreter Development REQUIRED)
include_directories(${Python3_INCLUDE_DIRS})

## Wrapper.Python
find_package(SWIG REQUIRED)
include(${SWIG_USE_FILE})
#include (UseSWIG)

set_property(SOURCE swigConfigurationFile.i PROPERTY CPLUSPLUS ON)
swig_add_library(SWIG_fileName LANGUAGE python SOURCES swigConfigurationFile.i)
swig_link_libraries(SWIG_fileName LibraryName ${Python3_LIBRARIES})

Writing CMakeListsFromPynb.txt


### Compiling

In [13]:
%%writefile RUNNER.py   
import os 
import numpy 
#print( numpy.get_include() ) 

os.environ['NP_INCLUDE_PATH'] = numpy.get_include()

os.system("rm -rf CMakeCache.txt CMakeLists.txt CMakeFiles cmake_install.cmake Make libLibraryName.*")  
os.system("cp CMakeListsFromPynb.txt CMakeLists.txt") 
os.system("cmake . -DCMAKE_CXX_COMPILER=g++ -DCMAKE_CXX_FLAGS=-I$NP_INCLUDE_PATH")  
os.system("make -B")  

Writing RUNNER.py


In [14]:
%%bash 
python RUNNER.py

-- The CXX compiler identification is GNU 9.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /opt/conda/bin/g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python3: /opt/conda/bin/python3.9 (found version "3.9.7") found components: Interpreter Development Development.Module Development.Embed 
-- Found SWIG: /opt/conda/bin/swig (found version "4.0.2")  
-- Configuring done
-- Generating done
-- Build files have been written to: /home/jovyan/work
[ 20%] Building CXX object CMakeFiles/LibraryName.dir/simpleExample.cpp.o
[ 40%] Linking CXX shared library libLibraryName.so
[ 40%] Built target LibraryName
Scanning dependencies of target SWIG_fileName_swig_compilation
[ 60%] Swig compile swigConfigurationFile.i for python
[ 60%] Built target SWIG_fileName_swig_compilation
[ 80%] Building CXX object CMakeFiles/SWIG_fileName.dir/CMakeFiles/SWIG_fileName.dir/swigConfigurationFileP

### Testing 

In [15]:
exec(open('test.py').read())

Hello from swig-example:
 [ 1 2 3 4 ] 
 [ 1 2 3 4 ] 


## Appendix

### A. Github Actions

Next files are automatically executed once they are located on *.github/workflows* 

#### A.1.  Building a Docker Image 

%%writefile buildImage.yml  
name: Building Docker Image

on:
  workflow_dispatch:

jobs:

  buildimage:
    runs-on: ubuntu-latest

    steps:
    - name: Building image ... 
      run: |
        echo "${{secrets.DOCKER_HUB_TOKEN}}" | docker login -u "${{secrets.DOCKER_HUB_USERNAME}}" --password-stdin docker.io
        
        docker build . --file dockerfile.build --tag docker.io/${{secrets.DOCKER_HUB_USERNAME}}/${{secrets.DOCKER_HUB_REPOSITORY}}:$GITHUB_SHA
            
        docker push docker.io/${{secrets.DOCKER_HUB_USERNAME}}/${{secrets.DOCKER_HUB_REPOSITORY}}:$GITHUB_SHA

#### A.2.  Executing a Docker Image 

%%writefile executeImage.yml  
name: Executing Docker Image

on: [push, workflow_dispatch]

jobs:
  executeimage:
    runs-on: ubuntu-latest

    steps:
    - name: OS
      run: uname -a

    - name: Downloading repository ...
      uses: actions/checkout@v2

    - name: Login to Docker hub ... 
      uses: actions-hub/docker/login@master
      env:
        DOCKER_USERNAME: ${{secrets.DOCKER_HUB_USERNAME}}
        DOCKER_PASSWORD: ${{secrets.DOCKER_HUB_TOKEN}}
            
    - name: Build
      if: success()
      run: |
        echo "[DOCKER] BUILDING ..."
        IMAGE_NAME=gh01
        docker build -f Dockerfile -t $IMAGE_NAME:1.0 .
        docker images

        echo "[DOCKER] CREATING ..." 
        CONTAINER_ID=$(docker create ${IMAGE_NAME}:1.0)

        echo "[DOCKER] EXECUTING ..."  
        docker cp ${CONTAINER_ID}:/home/jovyan/work/from_docker.ipynb ./
        docker rm -v $CONTAINER_ID
        
        echo "[DOCKER] DONE!" 

    - name: Saving artifacts ...
      uses: actions/upload-artifact@v2
      with:
        name: FromContainer
        if-no-files-found: error
        path: ./from_docker.ipynb
        retention-days: 1
        

Access tokens *DOCKER_HUB_TOKEN*, *DOCKER_HUB_REPOSITORY* and *DOCKER_HUB_USERNAME* have to be stored as sensitive information

### B. Launching interactive container processes

%%writefile docker.running_container 

IMAGE_NAME=
IMAGE_TAG=
CONTAINER_NAME=CONTAINER_${IMAGE_NAME}_${IMAGE_TAG}
CONTAINER_VOLUME=/SHARED
LOCAL_VOLUME=$(pwd)

echo "LOCAL_VOLUME:" $LOCAL_VOLUME
docker run --name $CONTAINER_NAME -v$LOCAL_VOLUME:$CONTAINER_VOLUME  --rm -it $IMAGE_NAME:$IMAGE_TAG /bin/bash

Now, execute 

bash docker.running_container

or, additionally

docker exec -it $CONTAINER_NAME /bin/bash

or

docker exec -e GRANT_SUDO=yes --user root -it $CONTAINER_NAME /bin/bash 

### C. Alternative to containerization

%%bash 
conda update -n base conda
conda install -c anaconda cmake swig numpy 